In [12]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import copy

video_path = '2022_08_25_ 000001_AFW.tif'


In [24]:
# Read video and decide on reference slice
video = sitk.ReadImage(video_path)

video_array = sitk.GetArrayFromImage(video)[:37]
number_of_slices = video_array.shape[0]

middle_slice = video_array[number_of_slices // 2]
middle_img = sitk.GetImageFromArray(middle_slice)

In [25]:
# Construct the elastix imag filter
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("rigid"))
elastixImageFilter.SetFixedImage(middle_img)

# Do registration for every third slice and save only the transformation parameters
transform_params = np.zeros((number_of_slices, 3))

for i in tqdm(range(0, len(video_array), 3), desc='Registering'):
    elastixImageFilter.SetMovingImage(sitk.GetImageFromArray(video_array[i]))
    elastixImageFilter.Execute()

    param_map = elastixImageFilter.GetTransformParameterMap()[0]
    transform_params[i] = ([float(i) for i in param_map['TransformParameters']])

center_of_rotation = np.array([float(i) for i in param_map['CenterOfRotationPoint']])
    

Registering:   0%|          | 0/13 [00:00<?, ?it/s]

In [27]:
# Extract two arrays of parameters, each with every sixth image filled and a faze shift of 3 images
first_transform_array = np.empty(transform_params.shape)
second_transform_array = np.empty(transform_params.shape)

first_transform_array[:] = np.nan
second_transform_array[:] = np.nan


for i in range(0, len(transform_params), 3):
    if i % 6 == 0:
        first_transform_array[i] = transform_params[i]
    else:
        second_transform_array[i] = transform_params[i]
    

# Interpolate in between the set parameters
def fill_out_array(array0, start, period):
    
    array = copy.deepcopy(array0)
    
    if start != 0:
        inter_array = np.zeros((start, 3))
        first = array[start]
        second = array[start + period]
        
        
        
        for j in range(start):
            inter_array[start - j - 1] = first - (second - first) * ((j+1) / period)
                
        array[:start] = inter_array
    
    for i in range(start, len(transform_params), period):
    
            
        first = array[i]
        second = array[i+period]
        inter_array = np.zeros((period - 1, 3))
        
        for j in range(period-1):
            inter_array[j] = first + (second - first) * ((j + 1) / period)
        
        array[i+1:i+period] = inter_array
        
        # To handle the end of the array
        if len(transform_params) - i == period + 1:
            break
        
        if len(transform_params) - i <= 2 * period:
            
            remainder = len(transform_params) - i - period - 1
            inter_array = np.zeros((remainder, 3))
            
            for j in range(remainder):
                inter_array[j] = second + (second - first) * ((j+1) / period)
                
            array[i+period + 1:] = inter_array
            
            break
        
    return array

# Execute interpolation and combine by taking the mean
first_transform_array = fill_out_array(first_transform_array, 0, 6)
second_transform_array = fill_out_array(second_transform_array, 3, 6)

total_transform_array = (first_transform_array + second_transform_array) / 2

# Now perform median-smoothing
smooth_transform_array = copy.deepcopy(total_transform_array)

for i in range(5, len(total_transform_array) - 5):
    smooth_transform_array[i] = np.median(total_transform_array[i-5:i+6], axis=0)

0 [-4.57184e-04 -2.78732e-01  5.19902e-01]
6 [-3.40015e-04 -2.10274e-01  3.79180e-01]

37
6 [-3.40015e-04 -2.10274e-01  3.79180e-01]
12 [-1.13317e-04 -8.27242e-02  1.63113e-01]

31
12 [-1.13317e-04 -8.27242e-02  1.63113e-01]
18 [-0.00014882  0.003367    0.0139237 ]

25
18 [-0.00014882  0.003367    0.0139237 ]
24 [ 0.00046787  0.064179   -0.142246  ]

19
24 [ 0.00046787  0.064179   -0.142246  ]
30 [ 0.00039423  0.132324   -0.28687   ]

13
30 [ 0.00039423  0.132324   -0.28687   ]
36 [ 3.74436e-04  2.35016e-01 -4.46391e-01]

just made it
3 [-3.20198e-04 -2.40269e-01  4.56010e-01]
9 [-1.37022e-04 -1.21006e-01  2.52279e-01]

34
9 [-1.37022e-04 -1.21006e-01  2.52279e-01]
15 [-9.51813e-05 -4.85781e-02  1.33713e-01]

28
15 [-9.51813e-05 -4.85781e-02  1.33713e-01]
21 [ 9.82092e-05  1.96145e-02 -4.55899e-02]

22
21 [ 9.82092e-05  1.96145e-02 -4.55899e-02]
27 [ 1.63105e-04  1.14123e-01 -2.20652e-01]

16
27 [ 1.63105e-04  1.14123e-01 -2.20652e-01]
33 [ 1.11010e-04  2.18522e-01 -3.62624e-01]

10
fi

In [29]:
second_transform_array

array([[-4.11786000e-04, -2.99900500e-01,  5.57875500e-01],
       [-3.81256667e-04, -2.80023333e-01,  5.23920333e-01],
       [-3.50727333e-04, -2.60146167e-01,  4.89965167e-01],
       [-3.20198000e-04, -2.40269000e-01,  4.56010000e-01],
       [-2.89668667e-04, -2.20391833e-01,  4.22054833e-01],
       [-2.59139333e-04, -2.00514667e-01,  3.88099667e-01],
       [-2.28610000e-04, -1.80637500e-01,  3.54144500e-01],
       [-1.98080667e-04, -1.60760333e-01,  3.20189333e-01],
       [-1.67551333e-04, -1.40883167e-01,  2.86234167e-01],
       [-1.37022000e-04, -1.21006000e-01,  2.52279000e-01],
       [-1.30048550e-04, -1.08934683e-01,  2.32518000e-01],
       [-1.23075100e-04, -9.68633667e-02,  2.12757000e-01],
       [-1.16101650e-04, -8.47920500e-02,  1.92996000e-01],
       [-1.09128200e-04, -7.27207333e-02,  1.73235000e-01],
       [-1.02154750e-04, -6.06494167e-02,  1.53474000e-01],
       [-9.51813000e-05, -4.85781000e-02,  1.33713000e-01],
       [-6.29495500e-05, -3.72126667e-02

In [ ]:
# Construct transform and resampler, then apply to video
out_array = np.zeros(video_array.shape)

transform = sitk.Euler2DTransform()
transform.SetCenter(center_of_rotation)

resampler = sitk.ResampleImageFilter()
resampler.SetInterpolator(sitk.sitkLinear)

for i in range(len(video_array)):
    transform.SetTranslation(smooth_transform_array[i][1:3])
    transform.SetAngle(smooth_transform_array[i][0])
    
    img = sitk.Resample(sitk.GetImageFromArray(video_array[i]),
                  middle_img, 
                  transform,
                  sitk.sitkLinear, 
                  0.0)
    out_array[i] = sitk.GetArrayFromImage(img)

In [ ]:
out_image = sitk.GetImageFromArray(out_array)
out_image.CopyInformation(video)

out_image = sitk.Cast(out_image, video.GetPixelID())
sitk.WriteImage(out_image, f'{video_path[:-4]}_fixed.tif')